# Hazard Map Generation

In [ ]:
import rasterio
import geopandas as gpd

# 1. Load the GeoTIFF file
tif_path = 'path_to_your_tif_file.tif'  # Replace with the actual file path

with rasterio.open(tif_path) as tif:
    tiff_data = tif.read(1)  # Read the first band of the TIFF file
    tiff_transform = tif.transform  # Get the transformation matrix of the TIFF
    tiff_crs = tif.crs  # Get the CRS (Coordinate Reference System)
    tiff_meta = tif.meta  # Get the metadata of the TIFF

print("TIFF data loaded:")
print(f"Shape: {tiff_data.shape}")
print(f"CRS: {tiff_crs}")

# 2. Load the Shapefile
shp_path = 'path_to_your_shapefile.shp'  # Replace with the actual file path
shapefile_data = gpd.read_file(shp_path)

# Optional: Ensure the shapefile has the same CRS as the TIFF
if shapefile_data.crs != tiff_crs:
    shapefile_data = shapefile_data.to_crs(tiff_crs)

print("Shapefile data loaded:")
print(f"Number of features: {len(shapefile_data)}")
print(f"CRS: {shapefile_data.crs}")


In [33]:
import numpy as np
import rasterio

# 1. Load the existing GeoTIFF to read its metadata and data
tif_path = 'D:/GISCUP/Hazard/Output_D_POI_with_Hazard.tif'

with rasterio.open(tif_path) as src:
    first_band = src.read(1)  # First band (original data)
    second_band = src.read(2)  # Second band (summed hazard values)
    transform = src.transform  # Transformation matrix
    crs = src.crs  # Coordinate Reference System (CRS)
    meta = src.meta  # Metadata
    width, height = src.width, src.height  # Dimensions of the TIFF

# 2. Calculate the threshold for the bottom 5% of the first band (excluding zero values)
non_zero_first_band = first_band[first_band > 0]  # Exclude 0 values

if non_zero_first_band.size > 0:
    first_band_threshold = np.quantile(non_zero_first_band, 0.10)  # Bottom 5% by quantity, excluding 0s
else:
    print("Warning: No non-zero values in first_band")
    first_band_threshold = 0  # Default or fallback value

# Create a mask for pixels in the first band that are in the bottom 5% (excluding 0s)
first_band_mask = (first_band > 0) & (first_band <= first_band_threshold)

# 3. Calculate the threshold for the top 5% of the second band (excluding zeros)
non_zero_second_band = second_band[second_band > 0]  # Exclude 0 values

if non_zero_second_band.size > 0:
    second_band_threshold = np.quantile(non_zero_second_band, 0.90)  # Top 5% by quantity
else:
    print("Warning: No non-zero values in second_band")
    second_band_threshold = 0  # Default or fallback value

# Create a mask for pixels in the second band that are in the top 5%
second_band_mask = (second_band > 0) & (second_band >= second_band_threshold)

# 4. Create the third band where masks are applied with distinct values
third_band = np.zeros_like(first_band, dtype=np.uint8)  # Initialize third band with zeros

# Set values based on mask conditions
third_band[first_band_mask & ~second_band_mask] = 100  # First band mask only
third_band[second_band_mask & ~first_band_mask] = 150  # Second band mask only
third_band[first_band_mask & second_band_mask] = 255   # Both masks overlap

# 5. Prepare metadata for writing all three bands
output_tif_path = 'D:/GISCUP/Hazard/Output_With_Three_Bands_Excluding_Zero.tif'

new_meta = meta.copy()
new_meta.update({
    'count': 3,  # Three bands: first, second, and third
    'dtype': 'uint8'  # Ensure the data type is correct
})

# 6. Write the new GeoTIFF file with all three bands
with rasterio.open(output_tif_path, 'w', **new_meta) as dst:
    dst.write(first_band, 1)  # Write the first band (original data)
    dst.write(second_band, 2)  # Write the second band (summed hazard values)
    dst.write(third_band, 3)  # Write the third band with distinct mask values

print(f"New GeoTIFF with three bands saved to {output_tif_path}")


New GeoTIFF with three bands saved to D:/GISCUP/Hazard/Output_With_Three_Bands_Excluding_Zero.tif
